In [ ]:
! pip install -q opendatasets sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 15.7 MB/s eta 0:00:00


In [ ]:
import opendatasets as od
import pandas as pd
import re
from sentence_transformers import SentenceTransformer
import numpy as np
from nltk.stem import WordNetLemmatizer
import nltk
from tqdm.notebook import tqdm
import faiss
import numpy as np

nltk.download('wordnet')

In [ ]:
! pip install -q opendatasets
import opendatasets as od
url = "https://www.kaggle.com/datasets/nandini1999/perfume-recommendation-dataset"
od.download(url)

Dataset URL: https://www.kaggle.com/datasets/nandini1999/perfume-recommendation-dataset


100%|██████████| 838k/838k [00:00<00:00, 572MB/s]

In [ ]:
path = "/content/perfume-recommendation-dataset/final_perfume_data.csv"
df = pd.read_csv(path, encoding='unicode_escape')

df = df.fillna("")
df = df.drop_duplicates()
print(df.info(), "\n", '-'*100)
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2191 entries, 0 to 2190
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name         2191 non-null   object
 1   Brand        2191 non-null   object
 2   Description  2191 non-null   object
 3   Notes        2191 non-null   object
 4   Image URL    2191 non-null   object
dtypes: object(5)
memory usage: 85.7+ KB
None 
 ----------------------------------------------------------------------------------------------------


,Name,Brand,Description,Notes,Image URL
count,2191,2191,2191,2191,2191
unique,2184,249,2167,2054,2191
top,New York Intense Eau de Parfum,TOM FORD Private Blend,In an era where modern fragrance can feel col...,,https://static.luckyscent.com/images/products/...
freq,2,39,2,80,1


In [ ]:
lemmatizer = WordNetLemmatizer()
embedding_model = SentenceTransformer('BAAI/bge-small-en-v1.5')

def clean_notes(notes):
    notes = re.split(r'[,/]', notes)
    notes = [lemmatizer.lemmatize(note.strip().lower()) for note in notes if note.strip()]
    return notes

def clean(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = text.lower()
    return text.strip()

def perfume_preprocessing(df):
    df['id'] = df.index
    cols = ['id'] + [col for col in df.columns if col != 'id']
    df = df[cols]
    df['Clean_Notes'] = df['Notes'].apply(clean_notes)
    df['Description'] = df['Description'].apply(clean)
    df["text"] = df.apply(lambda row: f"{row['Name']} by {row['Brand']}. {row['Description']}. Notes include: {row['Clean_Notes']}.", axis=1)
    return df

def add_embedding(df, embedding_model):
    print("Generating embeddings...")
    tqdm.pandas()
    df['embeddings'] = df['text'].progress_apply(lambda x: embedding_model.encode(x))
    print("Embeddings generated.")
    embeddings = np.vstack(df["embeddings"].values)
    np.save("/content/embeddings.npy", embeddings)
    return df

def get_final_dataset(df, embedding_model):
    df = perfume_preprocessing(df)
    df = add_embedding(df, embedding_model)
    return df

df = get_final_dataset(df, embedding_model)
df.to_csv("/content/perfume_dataset.csv", index=False)
df.head()

Generating embeddings...


<ipython-input-29-b996c9ff9e70>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Clean_Notes'] = df['Notes'].apply(clean_notes)
<ipython-input-29-b996c9ff9e70>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Description'] = df['Description'].apply(clean)
<ipython-input-29-b996c9ff9e70>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

  0%|          | 0/2191 [00:00<?, ?it/s]

<ipython-input-29-b996c9ff9e70>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['embeddings'] = df['text'].progress_apply(lambda x: embedding_model.encode(x))


Embeddings generated.


,id,Name,Brand,Description,Notes,Image URL,Clean_Notes,text,embeddings
0,0,Tihota Eau de Parfum,Indult,rapa nui for sugar tihota is quite simply the ...,"Vanilla bean, musks",https://static.luckyscent.com/images/products/...,"[vanilla bean, musk]",Tihota Eau de Parfum by Indult. rapa nui for s...,"[-0.0014193216, 0.04328476, 0.00011094599, -0...."
1,1,Sola Parfum,Di Ser,a tribute to the expanse of space extending fr...,"Lavender, Yuzu, Lemongrass, Magnolia, Geraniu...",https://static.luckyscent.com/images/products/...,"[lavender, yuzu, lemongrass, magnolia, geraniu...",Sola Parfum by Di Ser. a tribute to the expans...,"[0.002841696, 0.0068801483, 0.03676595, -0.002..."
2,2,Kagiroi Parfum,Di Ser,an aromatic ode to the ancient beauty of japan...,"Green yuzu, green shikuwasa, sansho seed, cor...",https://static.luckyscent.com/images/products/...,"[green yuzu, green shikuwasa, sansho seed, cor...",Kagiroi Parfum by Di Ser. an aromatic ode to t...,"[-0.01666905, 0.004298402, 0.015301916, 0.0201..."
3,3,Velvet Fantasy Eau de Parfum,Montale,velvet fantasy is a solar fragrance where citr...,"tangerine, pink pepper, black coffee, leat...",https://static.luckyscent.com/images/products/...,"[tangerine, pink pepper, black coffee, leather...",Velvet Fantasy Eau de Parfum by Montale. velve...,"[0.000368839, -0.00057079684, 0.01147244, 0.03..."
4,4,A Blvd. Called Sunset Eau de Parfum,A Lab on Fire,theres no way a lab on fire could relocate to ...,"Bergamot, almond, violet, jasmine, leather, s...",https://static.luckyscent.com/images/products/...,"[bergamot, almond, violet, jasmine, leather, s...",A Blvd. Called Sunset Eau de Parfum by A Lab o...,"[0.0013779446, 0.022209328, -0.023279507, 0.00..."


In [ ]:
d = embedding_model.get_sentence_embedding_dimension()

embeddings = np.vstack(df["embedding"].values)
faiss.normalize_L2(embeddings)

id_map = df.set_index("id").to_dict('index')

index = faiss.IndexIDMap(faiss.IndexFlatL2(d))
index.add_with_ids(embeddings, df['id'].values.astype('int64'))

In [ ]:
def infer(query, index, embedding_model, k=5):
    query_embedding = embedding_model.encode([query]).reshape(1, -1)
    faiss.normalize_L2(query_embedding)

    distances, indices = index.search(query_embedding, k)
    results = []
    for i, index in tqdm(enumerate(indices[0]), total=k):
        results.append({
            'id': index,
            'name': id_map[index]['Name'],
            'brand': id_map[index]['Brand'],
            'description': id_map[index]['Description'],
            'notes': id_map[index]['Clean_Notes'],
            'distance': distances[0][i],
            'image_url': id_map[index]['Image URL']
        })
    results = pd.DataFrame(results).set_index('id')
    return results

In [ ]:
results = infer("vanilla musk with a soft for winter", index, embedding_model, 6)

  0%|          | 0/6 [00:00<?, ?it/s]

description    exquisitely blended gardenia rose and vanilla ...
notes          [rose wardia, royal gardenia, frankincense, ve...
Name: 1802, dtype: object


In [ ]:
print(results["notes"].iloc[0],"\n", '-'*100)
print(results["description"].iloc[0])

['rose wardia', 'royal gardenia', 'frankincense', 'vetiver', 'madagascar vanilla', 'white musk'] 
 ----------------------------------------------------------------------------------------------------
exquisitely blended gardenia rose and vanilla flirt with the wearer and bring forth creamy shades of musk and frankincense  cotton musk is a study in the juxtaposition of seduction and purity  a potion that seeps gently into the skin until becomes a part of it


In [ ]:
import umap
import plotly.express as px

In [ ]:
reducer = umap.UMAP(n_neighbors=15, min_dist=0.1)
embedding_2d = reducer.fit_transform(embeddings)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [ ]:
df['x'] = embedding_2d[:, 0]
df['y'] = embedding_2d[:, 1]

fig = px.scatter(
    df, x='x', y='y',
    hover_data=['Name', 'Brand', 'Notes'],
    color='Brand',
    title="Perfume DNA Map"
)

fig.show()

<ipython-input-106-e4d0af8fb9d1>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-106-e4d0af8fb9d1>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

